In [1]:
import os
import rasterio
import numpy as np
import re



In [6]:
def extract_date(file_name):
    match = re.search(r'doy(\d{7})_', file_name)
    return match.group(1) if match else None

def normalize_band(band):
    band_min, band_max = band.min(), band.max()
    return (band - band_min) / (band_max - band_min)

def create_rgb(red_band_path, green_band_path, blue_band_path):
    with rasterio.open(red_band_path) as red_src, rasterio.open(green_band_path) as green_src, rasterio.open(blue_band_path) as blue_src:
        red_band = normalize_band(red_src.read(1))
        green_band = normalize_band(green_src.read(1))
        blue_band = normalize_band(blue_src.read(1))
        rgb = np.dstack((red_band, green_band, blue_band))
    return rgb, red_src.meta

def calculate_ndvi(nir_band_path, red_band_path):
    with rasterio.open(nir_band_path) as nir_src, rasterio.open(red_band_path) as red_src:
        nir_band = normalize_band(nir_src.read(1))
        red_band = normalize_band(red_src.read(1))
        np.seterr(divide='ignore', invalid='ignore')
        ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi, nir_src.meta

def calculate_ndwi(green_band_path, nir_band_path):
    with rasterio.open(green_band_path) as green_src, rasterio.open(nir_band_path) as nir_src:
        green_band = normalize_band(green_src.read(1).astype(float))
        nir_band = normalize_band(nir_src.read(1).astype(float))
        ndwi = np.divide(green_band - nir_band, green_band + nir_band, out=np.zeros_like(green_band), where=(green_band+nir_band)!=0)
    return ndwi, green_src.meta



def join_path(*args):
    return os.path.join(*args)

def save_true_color_to_tiff(true_color, src, output_filename):
    with rasterio.open(
        output_filename,
        'w',
        driver='GTiff',
        height=true_color.shape[0],
        width=true_color.shape[1],
        count=3,
        dtype=true_color.dtype,
        # crs=src.crs,
        # transform=src.transform
    ) as dst:
        for i in range(true_color.shape[2]):
            dst.write(true_color[:, :, i], i + 1)

def save_index_to_tiff(index, src, output_filename):
    with rasterio.open(
        output_filename,
        'w',
        driver='GTiff',
        height=index.shape[0],
        width=index.shape[1],
        count=1,
        dtype=index.dtype,
        # crs=src.crs,
        # transform=src.transform,
    ) as dst:
        dst.write(index, 1)


In [4]:


def process_landsat_files(source_folder):
    for root, dirs, files in os.walk(source_folder):
        # Group files by date
        date_files = {}
        for file in files:
            if file.endswith(".tif") and '_B' in file:
                date = extract_date(file)
                if date:
                    if date not in date_files:
                        date_files[date] = []
                    date_files[date].append(file)

        # Process each date
        for date, files in date_files.items():
            # Create a dictionary to map band numbers to file names using regex
            band_files = {}
            for file in files:
                match = re.search(r'_(B\d+)_', file)
                if match:
                    band_number = match.group(1)
                    band_files[band_number] = file

            if all(band in band_files for band in ['B1', 'B2', 'B3', 'B4']):
                nir_band_path = join_path(root, band_files['B4'])
                red_band_path = join_path(root, band_files['B3'])
                green_band_path = join_path(root, band_files['B2'])
                blue_band_path = join_path(root, band_files['B1'])
                
                ndvi, meta_ndvi = calculate_ndvi(nir_band_path, red_band_path)
                ndwi, meta_ndwi = calculate_ndwi(green_band_path, nir_band_path)
                rgb, meta_rgb = create_rgb(red_band_path, green_band_path, blue_band_path)

                # Save the indices and RGB image using the new functions
                output_folder_ndvi = join_path(root, 'NDVI')
                output_folder_ndwi = join_path(root, 'NDWI')
                output_folder_rgb = join_path(root, 'RGB')
                
                os.makedirs(output_folder_ndvi, exist_ok=True)
                os.makedirs(output_folder_ndwi, exist_ok=True)
                os.makedirs(output_folder_rgb, exist_ok=True)

                save_index_to_tiff(ndvi, meta_ndvi, join_path(output_folder_ndvi, f'NDVI_{date}.tiff'))
                save_index_to_tiff(ndwi, meta_ndwi, join_path(output_folder_ndwi, f'NDWI_{date}.tiff'))
                save_true_color_to_tiff(rgb, meta_rgb, join_path(output_folder_rgb, f'RGB_{date}.tiff'))


In [7]:
source_folder = os.path.join(os.getcwd(), "2023")
process_landsat_files(source_folder)

/Users/abolfazlyousefi/anaconda3/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


In [8]:
import pandas as pd

# Define the conditions based on the bit encoding
conditions = {
    0: 'Dark Dense Vegetation',
    1: 'Cloud',
    2: 'Cloud Shadow',
    3: 'Adjacent to Cloud',
    4: 'Snow',
    5: 'Water'
}

# Generate the table with binary representations and explanations
qa_values = range(41)
table = []

for value in qa_values:
    binary_str = format(value, '08b')
    flags = []
    for bit, condition in conditions.items():
        if value & (1 << bit):
            flags.append(f'Bit {bit} - {condition}')
    explanation = ' | '.join(flags) if flags else 'No flags set'
    table.append([value, binary_str, explanation])

# Create a DataFrame
df = pd.DataFrame(table, columns=['Value', 'Binary', 'Explanation'])

df.head() 

,Value,Binary,Explanation
0,0,00000000,No flags set
1,1,00000001,Bit 0 - Dark Dense Vegetation
2,2,00000010,Bit 1 - Cloud
3,3,00000011,Bit 0 - Dark Dense Vegetation | Bit 1 - Cloud
4,4,00000100,Bit 2 - Cloud Shadow


In [9]:
df

,Value,Binary,Explanation
0,0,00000000,No flags set
1,1,00000001,Bit 0 - Dark Dense Vegetation
2,2,00000010,Bit 1 - Cloud
3,3,00000011,Bit 0 - Dark Dense Vegetation | Bit 1 - Cloud
4,4,00000100,Bit 2 - Cloud Shadow
5,5,00000101,Bit 0 - Dark Dense Vegetation | Bit 2 - Cloud ...
6,6,00000110,Bit 1 - Cloud | Bit 2 - Cloud Shadow
7,7,00000111,Bit 0 - Dark Dense Vegetation | Bit 1 - Cloud ...
8,8,00001000,Bit 3 - Adjacent to Cloud
9,9,00001001,Bit 0 - Dark Dense Vegetation | Bit 3 - Adjace...
